In [86]:

import math
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils
import torch.utils.data
import torch.nn as nn
import torchsummary
import torch.nn.functional as F
import numpy as np
import random
import importlib
import gen_utils
importlib.reload(gen_utils)

<module 'gen_utils' from '/home/spometun/projects/solink/scripts/python/home_project/life/gen_utils.py'>

In [51]:
transform = transforms.Compose(
    [transforms.ToTensor()
        ,transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     ])
train=torchvision.datasets.CIFAR10("/home/spometun/datasets/research", train=True,
                                  download=True, transform=transform)
test=torchvision.datasets.CIFAR10("/home/spometun/datasets/research", train=False,
                                  download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [37]:
def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(2)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
# torchsummary.summary(net, (3, 32, 32), device=torch.device("cpu"))

In [88]:
d = [("a", 1), ("b", 2), ("c", 3)]
data, labels = gen_utils.split_dataset(d)

a = list(labels)
data[1]



'b'

In [47]:
print(3)

3


In [39]:
train_loader = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=4, shuffle=True)
def criterion(input, target):
    return nn.functional.cross_entropy(input, target) / math.log(2)

optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 1):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 0:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.544
[1,  4000] loss: 2.362
[1,  6000] loss: 2.225
[1,  8000] loss: 2.119
[1, 10000] loss: 2.062
[1, 12000] loss: 1.981
[2,  2000] loss: 1.855
[2,  4000] loss: 1.865
[2,  6000] loss: 1.844
[2,  8000] loss: 1.819
[2, 10000] loss: 1.791
[2, 12000] loss: 1.762
Finished Training
